This is a modified version of a code for building a 2D CNN model used for porosity estimation. 
The original code was made by Kurdistan Chawshin,  chawshinkurdistan@gmail.com, 2021

In [ ]:
import numpy as np 
seed = 7
np.random.seed(seed)
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split



import time
t0 = time.time()

loading the data. 

In [ ]:
aaaiTrainImages = np.load('../WholeData/WholeDataTrainImage.npy')
print(aaaiTrainImages.shape)

aaaiTestImages = np.load('../WholeData/WholeDataTestImage.npy')
print(aaaiTestImages.shape)

afTrainPermeability = np.load('../WholeData/WholeDataTrainPermeability.npy')
print(afTrainPermeability .shape)

afTestPermeability  = np.load('../WholeData/WholeDataTestPermeability.npy')

print(afTestPermeability .shape)


Train and validation sets split. 20% of the training set is used as the validation set. The validation set is used to evaluate the performance of the model during training.

In [ ]:
aaaiTrainImages, aaaiValidationImages, afTrainPermeability, afValidationPermeability  = train_test_split(aaaiTrainImages, afTrainPermeability , test_size=0.2, random_state=42, shuffle=True)

Image Augmentation: 
We rotate train images by three angles of 90, 180, and 270 degress. These rotated images together with the original images will be used for training. Note that these images are also flipped horizontally on the fly using "ImageDataGenerator" library.

In [ ]:
aaaiTrainImagesRot90 = np.rot90(aaaiTrainImages, axes=(1,2))

afTrainPermeability 90 = np.copy(afTrainPermeability)
aaaiTrainImagesRot180 = np.rot90(aaaiTrainImagesRot90, axes=(1,2))
afTrainPermeability180 = np.copy(afTrainPermeability)
aaaiTrainImagesRot270 = np.rot90(aaaiTrainImagesRot180, axes=(1,2))
afTrainPermeability270 = np.copy(afTrainPermeability)

aaaiTrainImages = np.concatenate((aaaiTrainImages, aaaiTrainImagesRot90, aaaiTrainImagesRot180, aaaiTrainImagesRot270), axis=0)
afTrainPermeability = np.concatenate((afTrainPermeability, afTrainPermeability90, afTrainPermeability180, afTrainPermeability270), axis=0)

print(aaaiTrainImages.shape)
print(afTrainPermeability.shape)

Define batch size and number of epochs for Keras Tuner

In [ ]:
batch_size = 32
epochs =30

convert each 64 x 64 image of the train and test set into a matrix of size 64 x 64 x 1 which is fed into the network. 1 is for number of channels. We are working with grayscale images. Therefore we have 1 channel. RGB images have 3 channels.

In [ ]:
XPix, YPix = aaaiTrainImages.shape[1], aaaiTrainImages.shape[2]
aaaiTrainImages = aaaiTrainImages.reshape(-1,XPix,YPix,1)
aaaiValidationImages = aaaiValidationImages.reshape(-1,XPix,YPix,1)
aaaiTestImages = aaaiTestImages.reshape(-1,XPix,YPix,1)


Print image shape for train, validation and test sets

In [ ]:
print ('Train Images shape with channel:', aaaiTrainImages.shape)
print ('Validation Images shape with channel:', aaaiValidationImages.shape)
print ('Test Images shape with channel:', aaaiTestImages.shape)
print ('Train Permeability shape:', afTrainPermeability.shape)
print ('Validation Permeability shape:', afValidationPermeability.shape)
print ('Test Permeability shape:', afTestPermeability.shape)


Image Augmenntation. Here we apply horizontal flip on the images that were already rotated by three degrees. We did the rotation outside Keras. 

In [ ]:
TrainDataGen = ImageDataGenerator(rescale= 1/255., horizontal_flip=True)
TrainDataGen.fit(aaaiTrainImages, augment=True)
TrainGenerator = TrainDataGen.flow(aaaiTrainImages, afTrainPermeability, batch_size=batch_size, seed=seed)


Rescale the pixel values between 0 and 1.

In [ ]:
aaaiValidationImages = aaaiValidationImages/255.
aaaiTestImages = aaaiTestImages/255.
print(np.min(aaaiTrainImages))

Import packages for CNN training

In [ ]:
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ReLU
from tensorflow.keras import activations
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

Define the model structure. Here we perform hyperparameter tuning useing Keras Tuner library. 
A range of values are defined for each of the considered hyperparameters. The algorithm will
consider different combinations and return the best configuration as the best performing model.

In [ ]:
def ModelCreation(hp):
    Regressor = keras.Sequential()
    
    Regressor.add(keras.layers.Conv2D(
            filters= hp.Int('ConvFiltersInputLayer', min_value = 16, max_value=256,step=32),
            kernel_size=hp.Choice('Conv2Kernel', values=[3,5]), padding='same',input_shape =(XPix, YPix, 1)))

    Regressor.add(keras.layers.ReLU())
    Regressor.add(keras.layers.MaxPooling2D(
            pool_size=(2, 2),padding='same'))
    
    for i in range(hp.Int('ConvBlocks', 1, 3, default=1)):
        Regressor.add(keras.layers.Conv2D(
            filters= hp.Int(f'Conv{i}_Filter', min_value = 16, max_value=256, step=32),
            kernel_size= hp.Choice('Conv2Kernel', values=[3,5]), padding='same'))
    
        Regressor.add(keras.layers.ReLU())
        Regressor.add(keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'))
        
    Regressor.add(keras.layers.Flatten())
        
    Regressor.add(keras.layers.Dense(
            units= hp.Int('HiddenNeurons', 32, 256, step=32),
            activation='linear'))
    Regressor.add(keras.layers.ReLU())
    
    Regressor.add(keras.layers.Dropout(hp.Float('dropout', min_value=0.0, max_value=0.6,default=0.00,step=0.2)))
    
    
    
    Regressor.add(keras.layers.Dense(1, activation='linear'))
    
    #Compile the model
    Regressor.compile(loss="mean_absolute_error", optimizer=keras.optimizers.Adam(
        hp.Choice('LearningRate',[1e-2, 1e-3, 1e-4])))
    return Regressor

            
    

In [ ]:
EarlyStop = EarlyStopping(monitor='val_loss', mode='min', patience=10)

Hyperband using keras-tuner

In [ ]:
Tuner = Hyperband(ModelCreation, objective= 'val_loss', max_epochs =epochs, executions_per_trial =1, seed=seed, directory=os.path.normpath('../codeOutputs/Permeability/TransportProp'), project_name='HyperUnRotValWholeDataMAEEarlyStopping10')

Tuner.search(TrainGenerator, verbose=2, validation_data= (aaaiValidationImages, afValidationPermeability), epochs=epochs, callbacks=[EarlyStop])

Tuner.search_space_summary()

Model = Tuner.get_best_models(1)[0]

print(Model.summary())

Tuner.results_summary()

print(Tuner.get_best_hyperparameters(1)[0].values)


The best returned model will train for more epochs.


In [ ]:
Epochs = 300
EarlyStop = EarlyStopping(monitor='val_loss', mode='min', patience=130)

In [29]:
experiment = "../CheckPointTest/TwoDArchitectureAugmentation/"

ckpt_pathname = experiment + "/cp-{epoch:04d}.ckpt"
csv_filename = experiment + "/metricsTwoDArchitectureAugmentation.csv"

Create a callback that saves the model's weights.

In [30]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_pathname, verbose=1)
metrics_callback = tf.keras.callbacks.CSVLogger(csv_filename, append=True)

In [ ]:
BestRegressor = Model.fit(TrainGenerator, validation_data = (aaaiValidationImages, afValidationPermeability), epochs=Epochs, callbacks =[EarlyStop], verbose=2)

This loads the specified checkpoint you want

In [32]:
#Model = keras.models.load_model(experiment + "cp-0005.ckpt")

Fits the model after loading the ckechpoints an initial epoch you want to start with (eg:stopped after epoch 10, but initial epoch at 10, because it starts couting from 0)

In [33]:
#BestRegressor1 = Model.fit(aaaiTrainImages, afTrainPermeability , validation_data = (aaaiValidationImages, afValidationPermeability), epochs=Epochs, initial_epoch=5, callbacks =[EarlyStop, cp_callback,metrics_callback], verbose=2)

Evaluate the model using test set

In [ ]:
t1 = time.time()
print('Training took: ',(t1 - t0)/60,'minutes')
TestLoss  = Model.evaluate(aaaiTestImages, afTestPermeability, verbose=2)
print('Test loss:', TestLoss)

Plot the improvement in loss function for the training and validation sets.

In [ ]:
plt.plot(BestRegressor.history['loss'], color ='b')
plt.plot(BestRegressor.history['val_loss'], color = 'r')
#plt.title('model loss')
plt.ylabel('MAE')
plt.xlabel('Epochs')
plt.legend(['Training loss', 'Validation loss'], loc='upper right')
plt.savefig('EpocsVsMSEWholeDataEarlystopping.png',dpi=1200, bbox_inches='tight')


Predict test set and save the model and predicted results.

In [ ]:
afPredictedPermeability = Model.predict(aaaiTestImages)
np.savetxt('../codeOutputs/Permeability/TransportProp/PredictedPermeability300EpochsMAEMoreConvUnrotatedValidationWholeDataMAEEarlyStopping10.csv', afPredicted)
save_model(Model, filepath= '../Models/PermeabilityEstimation/CNNModelForPermeabilityEstimation300EpochsMAEMoreConUnrotatedValidationWholeDataMAEEarlyStopping10', include_optimizer=True)